In [1]:
import cv2
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [21]:
# Cargar el modelo entrenado
model_path = '../models/reconocimiento_facial_15_personas.keras'
if not os.path.exists(model_path):
    raise FileNotFoundError(f"El modelo no se encontró en la ruta: {model_path}")
model = load_model(model_path)

# Clase de nombres de las 15 personas
class_names = ['David']  # Cambiar según corresponda

# Inicializar la cámara web
cap = cv2.VideoCapture(1)

# Cargar Haar Cascade para la detección de rostros
cascade_path = '../models/haarcascade_frontalface_alt.xml'
if not os.path.exists(cascade_path):
    raise FileNotFoundError(f"El clasificador Haar Cascade no se encontró en la ruta: {cascade_path}")
face_cascade = cv2.CascadeClassifier(cascade_path)

# Crear el archivo .csv si no existe
csv_file = '../results/asistencia.csv'
if not os.path.exists(csv_file):
    # Crear un DataFrame vacío con columnas "Nombre" y "Fecha"
    df = pd.DataFrame(columns=["Nombre", "Fecha"])
    df.to_csv(csv_file, index=False)

# Definir la función para registrar la asistencia
def registrar_asistencia(nombre):
    df = pd.read_csv(csv_file)
    fecha_actual = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    nuevo_registro = pd.DataFrame({"Nombre": [nombre], "Fecha": [fecha_actual]})
    df = pd.concat([df, nuevo_registro], ignore_index=True)
    df.to_csv(csv_file, index=False)

In [22]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("No se pudo capturar el frame de la cámara")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Dibujar un rectángulo alrededor del rostro
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Recortar el rostro de la imagen original
        face = frame[y:y+h, x:x+w]
        
        # Preprocesar el rostro
        face_resized = cv2.resize(face, (224, 224))  # Ajustar al tamaño que espera el modelo
        face_array = image.img_to_array(face_resized)
        face_array = np.expand_dims(face_array, axis=0)  # Añadir una dimensión para el lote
        
        # Normalizar el valor de los píxeles si es necesario (esto depende del preprocesamiento utilizado en el modelo)
        face_array /= 255.0
        
        # Verificar las dimensiones de la imagen
        if face_array.shape != (1, 224, 224, 3):
            print(f"Dimensiones incorrectas de la imagen: {face_array.shape}")
            continue
        
        # Realizar predicción
        try:
            predictions = model.predict(face_array)
        except Exception as e:
            print(f"Error al realizar la predicción: {e}")
            continue
        
        # Obtener el índice de la clase con la mayor probabilidad
        predicted_class = np.argmax(predictions)
        
        # Verificar que el índice de la clase esté dentro del rango
        if predicted_class >= len(class_names):
            print(f"Índice de clase fuera de rango: {predicted_class}")
            continue
        
        # Obtener el nombre de la persona y la precisión de la predicción
        person_name = class_names[predicted_class]
        confidence = np.max(predictions) * 100  # Convertir a porcentaje
        
        # Mostrar el nombre y la precisión sobre el rostro en el video
        label = f"{person_name}: {confidence:.2f}%"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Registrar la asistencia de la persona
        try:
            registrar_asistencia(person_name)
        except Exception as e:
            print(f"Error al registrar la asistencia: {e}")
    
    # Mostrar el video en tiempo real
    cv2.imshow('Reconocimiento Facial en Tiempo Real', frame)
    
    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()

d:\Ing. Sistemas y Computacion\Semestre VIII\Tópicos Especiales I\VA\Reconocimiento-Facial\Reconocimiento-Facial\VA\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━

Procesamiento del Dataframe

In [26]:
df = pd.read_csv(csv_file)

# Convertir la columna 'Fecha' a tipo datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Ordenar el DataFrame por 'Nombre' y 'Fecha'
df = df.sort_values(by=['Nombre', 'Fecha'])

# Crear un nuevo DataFrame para almacenar los registros procesados
df_processed = pd.DataFrame(columns=["Nombre", "Fecha"])

# Procesar el DataFrame para quedarse solo con la primera y última hora registrada de cada persona
for nombre in df['Nombre'].unique():
    df_persona = df[df['Nombre'] == nombre]
    primera_hora = df_persona.iloc[0]
    ultima_hora = df_persona.iloc[-1]
    df_processed = pd.concat([df_processed, pd.DataFrame([primera_hora])], ignore_index=True)
    if not primera_hora.equals(ultima_hora):
        df_processed = pd.concat([df_processed, pd.DataFrame([ultima_hora])], ignore_index=True)

# Guardar el DataFrame procesado en un nuevo archivo CSV
df_processed.to_csv('../results/asistencia.csv', index=False)

print("El archivo 'asistencia.csv' ha sido procesado con éxito.")

El archivo 'asistencia.csv' ha sido procesado con éxito.


C:\Users\rodfe\AppData\Local\Temp\ipykernel_18188\3066198800.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_processed = pd.concat([df_processed, pd.DataFrame([primera_hora])], ignore_index=True)
